# Week 3: Deep Learning Forecasting and Visualization
## US Collisions (2016–2023): Neural Sequence Models
### Mario Cuevas

---

**Objectives:**
- Implement Bidirectional LSTM/GRU models
- Build Temporal Convolutional Networks (TCN)
- Develop Transformer-based attention models
- Create attention visualizations for interpretability
- Perform cross-validation on state-level splits

---

In [1]:
print("📦 Installing deep learning packages...\n")

!pip install torch torchvision torchaudio --quiet
!pip install tensorflow --quiet
!pip install optuna --quiet
!pip install shap --quiet
!pip install plotly --quiet
!pip install kaleido --quiet

print("✅ Installation complete!")

📦 Installing deep learning packages...

✅ Installation complete!


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Deep Learning Frameworks
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks

# Model evaluation
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Visualization
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
tf.random.set_seed(42)

print("✅ Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"TensorFlow version: {tf.__version__}")

✅ Libraries imported successfully!
PyTorch version: 2.8.0+cu126
TensorFlow version: 2.19.0


## 1. Load Preprocessed Data from Week 2

In [3]:
# Load the feature-engineered dataset from Week 2
print("=" * 60)
print("LOADING PREPROCESSED DATA FROM WEEK 2")
print("=" * 60)

# Load daily aggregated data
daily_accidents = pd.read_csv('/content/daily_accidents_features.csv',
                              parse_dates=['Date'], index_col=0)

print(f"\n✅ Loaded {len(daily_accidents):,} daily records")
print(f"Date range: {daily_accidents.index.min()} to {daily_accidents.index.max()}")
print(f"\nFeatures available: {list(daily_accidents.columns)}")

daily_accidents.head()

LOADING PREPROCESSED DATA FROM WEEK 2

✅ Loaded 2,568 daily records
Date range: 2016-01-14 00:00:00 to 2023-03-31 00:00:00

Features available: ['Accident_Count', 'Avg_Severity', 'Avg_Temperature', 'Avg_Humidity', 'Avg_Visibility', 'Avg_Wind_Speed', 'Avg_Precipitation', 'Accident_Count_7d_MA', 'Accident_Count_30d_MA', 'Accident_Count_7d_Std', 'Accident_Count_Lag_1', 'Accident_Count_Lag_3', 'Accident_Count_Lag_7', 'Accident_Count_Lag_14', 'Accident_Count_Lag_30', 'Avg_Temperature_Lag_1', 'Avg_Humidity_Lag_1', 'Avg_Visibility_Lag_1', 'Avg_Temperature_Lag_7', 'Avg_Humidity_Lag_7', 'Avg_Visibility_Lag_7', 'Weather_Risk_Index']


,Accident_Count,Avg_Severity,Avg_Temperature,Avg_Humidity,Avg_Visibility,Avg_Wind_Speed,Avg_Precipitation,Accident_Count_7d_MA,Accident_Count_30d_MA,Accident_Count_7d_Std,...,Accident_Count_Lag_7,Accident_Count_Lag_14,Accident_Count_Lag_30,Avg_Temperature_Lag_1,Avg_Humidity_Lag_1,Avg_Visibility_Lag_1,Avg_Temperature_Lag_7,Avg_Humidity_Lag_7,Avg_Visibility_Lag_7,Weather_Risk_Index
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-14,7,4.000000,31.000000,69.000000,10.000000,3.000000,0.000000,7.00,7.00,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.252220
2016-02-08,60,2.316667,35.388333,91.816667,7.130000,6.681667,0.015000,33.50,33.50,37.476659,...,NaN,NaN,NaN,31.000000,69.000000,10.000000,NaN,NaN,NaN,1.527727
2016-02-09,59,2.542373,23.532203,87.118644,2.410169,11.461017,0.005932,42.00,42.00,30.315013,...,NaN,NaN,NaN,35.388333,91.816667,7.130000,NaN,NaN,NaN,3.596967
2016-02-10,49,2.510204,17.765306,77.571429,3.875510,12.767347,0.005306,43.75,43.75,24.998333,...,NaN,NaN,NaN,23.532203,87.118644,2.410169,NaN,NaN,NaN,2.900429
2016-02-11,93,2.354839,15.888172,65.311828,9.263441,8.975269,0.007204,53.60,53.60,30.883653,...,NaN,NaN,NaN,17.765306,77.571429,3.875510,NaN,NaN,NaN,-0.015010


## 2. Prepare Time Series Data for Deep Learning

We'll create sequences with a lookback window for the models.

In [4]:
def create_sequences(data, target_col, feature_cols, lookback=30, forecast_horizon=7):
    """
    Create sequences for time series forecasting.

    Args:
        data: DataFrame with time series data
        target_col: Name of target column
        feature_cols: List of feature column names
        lookback: Number of past timesteps to use
        forecast_horizon: Number of future timesteps to predict

    Returns:
        X: Input sequences [samples, lookback, features]
        y: Target values [samples, forecast_horizon]
    """
    X, y = [], []

    data_values = data[feature_cols].values
    target_values = data[target_col].values

    for i in range(len(data) - lookback - forecast_horizon + 1):
        X.append(data_values[i:i+lookback])
        y.append(target_values[i+lookback:i+lookback+forecast_horizon])

    return np.array(X), np.array(y)

print("=" * 60)
print("PREPARING SEQUENCES FOR DEEP LEARNING")
print("=" * 60)

# Select features
feature_cols = ['Accident_Count', 'Avg_Temperature', 'Avg_Humidity', 'Avg_Visibility',
                'Accident_Count_7d_MA', 'Accident_Count_30d_MA',
                'Accident_Count_Lag_1', 'Accident_Count_Lag_7', 'Accident_Count_Lag_30', 'Weather_Risk_Index']

target_col = 'Accident_Count'

df_clean = daily_accidents.dropna(subset=feature_cols + [target_col])

print(f"\nClean dataset: {len(df_clean):,} rows")

# Normalize features
scaler_X = StandardScaler()
scaler_y = StandardScaler()

df_clean[feature_cols] = scaler_X.fit_transform(df_clean[feature_cols])
df_clean[target_col] = scaler_y.fit_transform(df_clean[[target_col]])

# Create sequences
LOOKBACK = 14
FORECAST_HORIZON = 1

X, y = create_sequences(df_clean, target_col, feature_cols,
                        lookback=LOOKBACK, forecast_horizon=FORECAST_HORIZON)

print(f"\n✅ Created sequences:")
print(f"   X shape: {X.shape} (samples, lookback, features)")
print(f"   y shape: {y.shape} (samples, forecast_horizon)")

# Train/validation/test split (60/20/20)
train_size = int(0.6 * len(X))
val_size = int(0.2 * len(X))

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

print(f"\nTrain: {X_train.shape[0]} samples")
print(f"Val:   {X_val.shape[0]} samples")
print(f"Test:  {X_test.shape[0]} samples")

PREPARING SEQUENCES FOR DEEP LEARNING

Clean dataset: 2,538 rows

✅ Created sequences:
   X shape: (2524, 14, 10) (samples, lookback, features)
   y shape: (2524, 1) (samples, forecast_horizon)

Train: 1514 samples
Val:   504 samples
Test:  506 samples


## 3. Model 1: Bidirectional LSTM with Attention

In [5]:
# Build Bidirectional LSTM with Attention
print("=" * 60)
print("BUILDING BIDIRECTIONAL LSTM MODEL")
print("=" * 60)

class AttentionLayer(layers.Layer):
    """Custom attention layer for interpretability"""
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight',
                                shape=(input_shape[-1], input_shape[-1]),
                                initializer='glorot_uniform',
                                trainable=True)
        self.b = self.add_weight(name='attention_bias',
                                shape=(input_shape[-1],),
                                initializer='zeros',
                                trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        e = tf.nn.tanh(tf.tensordot(inputs, self.W, axes=1) + self.b)
        a = tf.nn.softmax(e, axis=1)

        output = inputs * a
        return tf.reduce_sum(output, axis=1)

def build_lstm_model(input_shape, output_steps):
    """Build Bidirectional LSTM with Attention"""
    inputs = layers.Input(shape=input_shape)

    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(inputs)
    x = layers.Dropout(0.3)(x)

    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
    x = layers.Dropout(0.3)(x)

    # Attention mechanism
    x = AttentionLayer()(x)

    # Dense layers
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.2)(x)

    outputs = layers.Dense(output_steps)(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# Build and compile model
lstm_model = build_lstm_model(input_shape=(LOOKBACK, len(feature_cols)),
                              output_steps=FORECAST_HORIZON)

lstm_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                   loss='mse',
                   metrics=['mae'])

print("\n✅ LSTM Model built successfully!")
lstm_model.summary()

BUILDING BIDIRECTIONAL LSTM MODEL

✅ LSTM Model built successfully!


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 14, 10)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 14, 256)        │       142,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 14, 128)        │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 14, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_layer                 │ (None, 128)            │        16,512 │
│ (AttentionLayer)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 331,521 (1.26 MB)

 Trainable params: 331,521 (1.26 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Train LSTM model
print("=" * 60)
print("TRAINING LSTM MODEL")
print("=" * 60)

early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=10,
                                     restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                        patience=5, min_lr=1e-6)

history_lstm = lstm_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,  # ✅ QUICK FIX #3: Increased for better convergence
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# Evaluate on test set
lstm_pred = lstm_model.predict(X_test)

# ✅ QUICK FIX #4: Denormalize predictions for accurate MAE
lstm_pred_denorm = scaler_y.inverse_transform(lstm_pred)
y_test_denorm = scaler_y.inverse_transform(y_test)

lstm_mae = mean_absolute_error(y_test_denorm.flatten(), lstm_pred_denorm.flatten())
lstm_rmse = np.sqrt(mean_squared_error(y_test_denorm.flatten(), lstm_pred_denorm.flatten()))
lstm_r2 = r2_score(y_test_denorm.flatten(), lstm_pred_denorm.flatten())

print(f"\n=== LSTM Test Results ===")
print(f"MAE:  {lstm_mae:.4f}")
print(f"RMSE: {lstm_rmse:.4f}")
print(f"R²:   {lstm_r2:.4f}")

TRAINING LSTM MODEL
Epoch 1/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 15s 55ms/step - loss: 0.5525 - mae: 0.6400 - val_loss: 1.0581 - val_mae: 0.7862 - learning_rate: 0.0010
Epoch 2/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 0.3679 - mae: 0.5023 - val_loss: 1.4048 - val_mae: 0.8295 - learning_rate: 0.0010
Epoch 3/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.2476 - mae: 0.3612 - val_loss: 1.4031 - val_mae: 0.7879 - learning_rate: 0.0010
Epoch 4/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.1828 - mae: 0.3001 - val_loss: 1.1239 - val_mae: 0.7281 - learning_rate: 0.0010
Epoch 5/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.1732 - mae: 0.2871 - val_loss: 1.0134 - val_mae: 0.7008 - learning_rate: 0.0010
Epoch 6/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.1576 - mae: 0.2704 - val_loss: 0.6680 - val_mae: 0.5758 - learning_rate: 0.0010
Epoch 7/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.1549 - mae: 0.2636 - val_loss: 0.6743 - val_mae: 0.5787 - learnin

## 4. Model 2: GRU Network

In [7]:
# ✅ QUICK FIX #5: Simplified GRU architecture (prevent overfitting)
print("=" * 60)
print("BUILDING SIMPLIFIED GRU MODEL")
print("=" * 60)

def build_gru_model(input_shape, output_steps):
    """Build Simplified Bidirectional GRU model"""
    inputs = layers.Input(shape=input_shape)

    # Single Bidirectional GRU layer (simplified from 2 layers)
    x = layers.Bidirectional(layers.GRU(64, return_sequences=False))(inputs)
    x = layers.Dropout(0.2)(x)  # Apply dropout to the output of the GRU layer

    # Single dense layer
    x = layers.Dense(32, activation='relu')(x)  # Reduced from 64
    x = layers.Dropout(0.1)(x)  # Apply dropout to the output of the Dense layer

    outputs = layers.Dense(output_steps)(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model

gru_model = build_gru_model(input_shape=(LOOKBACK, len(feature_cols)),
                            output_steps=FORECAST_HORIZON)

gru_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae'])

print("\n✅ Simplified GRU Model built successfully!")
print("Changes: 2 layers → 1 layer, 128/64 units → 64 units")
gru_model.summary()

BUILDING SIMPLIFIED GRU MODEL

✅ Simplified GRU Model built successfully!
Changes: 2 layers → 1 layer, 128/64 units → 64 units


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 14, 10)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ (None, 128)            │        29,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,345 (130.25 KB)

 Trainable params: 33,345 (130.25 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Train GRU model
print("=" * 60)
print("TRAINING GRU MODEL")
print("=" * 60)

history_gru = gru_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,  # ✅ QUICK FIX #3: Increased for better convergence
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# Evaluate
gru_pred = gru_model.predict(X_test)

# ✅ QUICK FIX #4: Denormalize predictions for accurate MAE
gru_pred_denorm = scaler_y.inverse_transform(gru_pred)
y_test_denorm = scaler_y.inverse_transform(y_test)

gru_mae = mean_absolute_error(y_test_denorm.flatten(), gru_pred_denorm.flatten())
gru_rmse = np.sqrt(mean_squared_error(y_test_denorm.flatten(), gru_pred_denorm.flatten()))
gru_r2 = r2_score(y_test_denorm.flatten(), gru_pred_denorm.flatten())

print(f"\n=== GRU Test Results ===")
print(f"MAE:  {gru_mae:.4f}")
print(f"RMSE: {gru_rmse:.4f}")
print(f"R²:   {gru_r2:.4f}")

TRAINING GRU MODEL
Epoch 1/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 0.4501 - mae: 0.5553 - val_loss: 0.6702 - val_mae: 0.6065 - learning_rate: 0.0010
Epoch 2/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2126 - mae: 0.3336 - val_loss: 0.5493 - val_mae: 0.5379 - learning_rate: 0.0010
Epoch 3/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.1592 - mae: 0.2767 - val_loss: 0.4978 - val_mae: 0.5099 - learning_rate: 0.0010
Epoch 4/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.1545 - mae: 0.2714 - val_loss: 0.5373 - val_mae: 0.5342 - learning_rate: 0.0010
Epoch 5/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.1572 - mae: 0.2720 - val_loss: 0.4773 - val_mae: 0.4979 - learning_rate: 0.0010
Epoch 6/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.1378 - mae: 0.2516 - val_loss: 0.5322 - val_mae: 0.5266 - learning_rate: 0.0010
Epoch 7/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1388 - mae: 0.2603 - val_loss: 0.5020 - val_mae: 0.5048 - learning_ra

## 5. Model 3: Temporal Convolutional Network (TCN)

In [9]:
# Build TCN model
print("=" * 60)
print("BUILDING TEMPORAL CONVOLUTIONAL NETWORK (TCN)")
print("=" * 60)

def build_tcn_model(input_shape, output_steps):
    """Build Temporal Convolutional Network"""
    inputs = layers.Input(shape=input_shape)

    # TCN blocks with dilated causal convolutions
    x = inputs
    for dilation_rate in [1, 2, 4, 8]:
        # Dilated causal convolution
        conv = layers.Conv1D(filters=64, kernel_size=3,
                           dilation_rate=dilation_rate,
                           padding='causal', activation='relu')(x)
        conv = layers.Dropout(0.2)(conv)

        # Residual connection
        if x.shape[-1] != conv.shape[-1]:
            x = layers.Conv1D(filters=64, kernel_size=1)(x)
        x = layers.Add()([x, conv])

    # Global pooling and dense layers
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation='relu')(x)

    outputs = layers.Dense(output_steps)(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model

tcn_model = build_tcn_model(input_shape=(LOOKBACK, len(feature_cols)),
                            output_steps=FORECAST_HORIZON)

tcn_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae'])

print("\n✅ TCN Model built successfully!")
tcn_model.summary()

BUILDING TEMPORAL CONVOLUTIONAL NETWORK (TCN)

✅ TCN Model built successfully!


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 14, 10)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 14, 64)    │      1,984 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 14, 64)    │        704 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 14, 64)    │          0 │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 14, 64)    │          0 │ conv1d_1[0][0],   │
│                     │                   │            │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 14, 64)    │     12,352 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 14, 64)    │          0 │ conv1d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 14, 64)    │          0 │ add[0][0],        │
│                     │                   │            │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 14, 64)    │     12,352 │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 14, 64)    │          0 │ conv1d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 14, 64)    │          0 │ add_1[0][0],      │
│                     │                   │            │ dropout_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 14, 64)    │     12,352 │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 14, 64)    │          0 │ conv1d_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 14, 64)    │          0 │ add_2[0][0],      │
│                     │                   │            │ dropout_8[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ add_3[0][0]       │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 128)       │      8,320 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 128)       │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 64)        │      8,256 │ dropout_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 1)         │         65 │ dense_5[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 56,385 (220.25 KB)

 Trainable params: 56,385 (220.25 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Train TCN model
print("=" * 60)
print("TRAINING TCN MODEL")
print("=" * 60)

history_tcn = tcn_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,  # ✅ QUICK FIX #3: Increased for better convergence
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# Evaluate
tcn_pred = tcn_model.predict(X_test)

# ✅ QUICK FIX #4: Denormalize predictions for accurate MAE
tcn_pred_denorm = scaler_y.inverse_transform(tcn_pred)
y_test_denorm = scaler_y.inverse_transform(y_test)

tcn_mae = mean_absolute_error(y_test_denorm.flatten(), tcn_pred_denorm.flatten())
tcn_rmse = np.sqrt(mean_squared_error(y_test_denorm.flatten(), tcn_pred_denorm.flatten()))
tcn_r2 = r2_score(y_test_denorm.flatten(), tcn_pred_denorm.flatten())

print(f"\n=== TCN Test Results ===")
print(f"MAE:  {tcn_mae:.4f}")
print(f"RMSE: {tcn_rmse:.4f}")
print(f"R²:   {tcn_r2:.4f}")

TRAINING TCN MODEL
Epoch 1/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 12s 109ms/step - loss: 0.6139 - mae: 0.6547 - val_loss: 0.9827 - val_mae: 0.7718 - learning_rate: 0.0010
Epoch 2/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4831 - mae: 0.5925 - val_loss: 1.0204 - val_mae: 0.7487 - learning_rate: 0.0010
Epoch 3/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3761 - mae: 0.4983 - val_loss: 0.8610 - val_mae: 0.6734 - learning_rate: 0.0010
Epoch 4/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2647 - mae: 0.3923 - val_loss: 0.7821 - val_mae: 0.6391 - learning_rate: 0.0010
Epoch 5/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2187 - mae: 0.3531 - val_loss: 0.8614 - val_mae: 0.6651 - learning_rate: 0.0010
Epoch 6/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2013 - mae: 0.3292 - val_loss: 0.6897 - val_mae: 0.6012 - learning_rate: 0.0010
Epoch 7/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1836 - mae: 0.3067 - val_loss: 0.6468 - val_mae: 0.5833 - learning_rate

## 6. Model 4: Transformer with Multi-Head Attention

In [11]:
# Build Transformer model
print("=" * 60)
print("BUILDING TRANSFORMER MODEL")
print("=" * 60)

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    """Single Transformer encoder block"""
    # Multi-head attention
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = layers.Add()([x, inputs])

    # Feed-forward network
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation='relu')(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return layers.Add()([x, res])

def build_transformer_model(input_shape, output_steps,
                           head_size=64, num_heads=4, ff_dim=128,
                           num_transformer_blocks=2, dropout=0.2):
    """Build Transformer model for time series forecasting"""
    inputs = layers.Input(shape=input_shape)

    # Initial projection
    x = layers.Dense(head_size * num_heads)(inputs)

    # Transformer encoder blocks
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    # Global pooling and output layers
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(64, activation='relu')(x)

    outputs = layers.Dense(output_steps)(x)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model

transformer_model = build_transformer_model(
    input_shape=(LOOKBACK, len(feature_cols)),
    output_steps=FORECAST_HORIZON,
    head_size=64,
    num_heads=4,
    ff_dim=128,
    num_transformer_blocks=2,
    dropout=0.2
)

transformer_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

print("\n✅ Transformer Model built successfully!")
transformer_model.summary()

BUILDING TRANSFORMER MODEL

✅ Transformer Model built successfully!


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 14, 10)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 14, 256)   │      2,816 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 14, 256)   │    263,168 │ dense_7[0][0],    │
│ (MultiHeadAttentio… │                   │            │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 14, 256)   │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 14, 256)   │        512 │ dropout_11[0][0]  │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 14, 256)   │          0 │ layer_normalizat… │
│                     │                   │            │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 14, 128)   │     32,896 │ add_4[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_12          │ (None, 14, 128)   │          0 │ conv1d_5[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_6 (Conv1D)   │ (None, 14, 256)   │     33,024 │ dropout_12[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 14, 256)   │        512 │ conv1d_6[0][0]    │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (None, 14, 256)   │          0 │ layer_normalizat… │
│                     │                   │            │ add_4[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 14, 256)   │    263,168 │ add_5[0][0],      │
│ (MultiHeadAttentio… │                   │            │ add_5[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_14          │ (None, 14, 256)   │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 14, 256)   │        512 │ dropout_14[0][0]  │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_6 (Add)         │ (None, 14, 256)   │          0 │ layer_normalizat… │
│                     │                   │            │ add_5[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_7 (Conv1D)   │ (None, 14, 128)   │     32,896 │ add_6[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_15          │ (None, 14, 128)   │          0 │ conv1d_7[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_8 (Conv1D)   │ (None, 14, 256)   │     33,024 │ dropout_15[0][0]  │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 704,257 (2.69 MB)

 Trainable params: 704,257 (2.69 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Train Transformer model
print("=" * 60)
print("TRAINING TRANSFORMER MODEL")
print("=" * 60)

history_transformer = transformer_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,  # ✅ QUICK FIX #3: Increased for better convergence
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# Evaluate
transformer_pred = transformer_model.predict(X_test)

# ✅ QUICK FIX #4: Denormalize predictions for accurate MAE
transformer_pred_denorm = scaler_y.inverse_transform(transformer_pred)
y_test_denorm = scaler_y.inverse_transform(y_test)

transformer_mae = mean_absolute_error(y_test_denorm.flatten(), transformer_pred_denorm.flatten())
transformer_rmse = np.sqrt(mean_squared_error(y_test_denorm.flatten(), transformer_pred_denorm.flatten()))
transformer_r2 = r2_score(y_test_denorm.flatten(), transformer_pred_denorm.flatten())

print(f"\n=== Transformer Test Results ===")
print(f"MAE:  {transformer_mae:.4f}")
print(f"RMSE: {transformer_rmse:.4f}")
print(f"R²:   {transformer_r2:.4f}")

TRAINING TRANSFORMER MODEL
Epoch 1/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 24s 217ms/step - loss: 4.0111 - mae: 1.3353 - val_loss: 1.4127 - val_mae: 0.9085 - learning_rate: 0.0010
Epoch 2/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.6646 - mae: 0.6817 - val_loss: 1.2668 - val_mae: 0.8759 - learning_rate: 0.0010
Epoch 3/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5862 - mae: 0.6619 - val_loss: 1.2908 - val_mae: 0.8770 - learning_rate: 0.0010
Epoch 4/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5726 - mae: 0.6620 - val_loss: 1.2994 - val_mae: 0.8832 - learning_rate: 0.0010
Epoch 5/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5595 - mae: 0.6444 - val_loss: 1.2640 - val_mae: 0.8687 - learning_rate: 0.0010
Epoch 6/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5822 - mae: 0.6569 - val_loss: 1.1707 - val_mae: 0.8499 - learning_rate: 0.0010
Epoch 7/100
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.5710 - mae: 0.6512 - val_loss: 1.1902 - val_mae: 0.8424 - learn

## 7. Model Comparison and Results Summary

In [13]:
# Compare all models
print("=" * 80)
print("MODEL COMPARISON SUMMARY")
print("=" * 80)

results_df = pd.DataFrame({
    'Model': ['LSTM + Attention', 'GRU', 'TCN', 'Transformer'],
    'MAE': [lstm_mae, gru_mae, tcn_mae, transformer_mae],
    'RMSE': [lstm_rmse, gru_rmse, tcn_rmse, transformer_rmse],
    'R²': [lstm_r2, gru_r2, tcn_r2, transformer_r2]
})

results_df = results_df.sort_values('MAE')
print("\n", results_df.to_string(index=False))

# Visualize comparison
fig = go.Figure()

# MAE comparison
fig.add_trace(go.Bar(
    name='MAE',
    x=results_df['Model'],
    y=results_df['MAE'],
    marker_color='lightblue'
))

fig.update_layout(
    title='Deep Learning Models: MAE Comparison',
    xaxis_title='Model',
    yaxis_title='Mean Absolute Error',
    template='plotly_white',
    height=400
)

fig.show()

# Save results
results_df.to_csv('/content/model_comparison_results.csv', index=False)
print("\n✅ Results saved to 'model_comparison_results.csv'")

MODEL COMPARISON SUMMARY

            Model      MAE     RMSE       R²
             GRU 0.560138 0.771053 0.415358
LSTM + Attention 0.635911 0.845614 0.296821
             TCN 0.649819 0.901289 0.201178
     Transformer 0.807704 0.994722 0.026973



✅ Results saved to 'model_comparison_results.csv'


## 8. Attention Visualization and Interpretability

In [14]:
# Extract attention weights from LSTM model
print("=" * 60)
print("ATTENTION VISUALIZATION")
print("=" * 60)

# Create a model that outputs attention weights
attention_model = models.Model(
    inputs=lstm_model.input,
    outputs=[lstm_model.output, lstm_model.layers[-4].output]  # Attention layer output
)

# Get predictions and attention weights
sample_idx = 100
sample_input = X_test[sample_idx:sample_idx+1]
pred, attention_output = attention_model.predict(sample_input)

print(f"\n✅ Extracted attention weights for sample {sample_idx}")
print(f"   Attention output shape: {attention_output.shape}")

ATTENTION VISUALIZATION
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step

✅ Extracted attention weights for sample 100
   Attention output shape: (1, 128)


In [15]:
# Visualize feature importance over time using attention
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Feature Values Over Time', 'Attention Weights'),
    vertical_spacing=0.15
)

# Plot input features
for i, feature_name in enumerate(feature_cols[:5]):  # Show top 5 features
    fig.add_trace(
        go.Scatter(
            x=list(range(LOOKBACK)),
            y=sample_input[0, :, i],
            name=feature_name,
            mode='lines'
        ),
        row=1, col=1
    )

# Plot attention heatmap
fig.add_trace(
    go.Heatmap(
        z=attention_output[0].reshape(1, -1),
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title='Attention')
    ),
    row=2, col=1
)

fig.update_layout(
    title='Attention Mechanism Visualization',
    height=700,
    template='plotly_white'
)

fig.update_xaxes(title_text='Time Step', row=1, col=1)
fig.update_xaxes(title_text='Time Step', row=2, col=1)
fig.update_yaxes(title_text='Normalized Value', row=1, col=1)

fig.show()

## 9. Prediction Visualization: Actual vs Predicted

In [16]:
# Visualize predictions
print("=" * 60)
print("FORECAST VISUALIZATION")
print("=" * 60)

# Get predictions for all models on test set
test_samples = 50  # Show 50 test samples

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('LSTM + Attention', 'GRU', 'TCN', 'Transformer'),
    vertical_spacing=0.15,
    horizontal_spacing=0.1
)

models_pred = [
    (lstm_pred, 'LSTM'),
    (gru_pred, 'GRU'),
    (tcn_pred, 'TCN'),
    (transformer_pred, 'Transformer')
]

positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

for (pred, name), (row, col) in zip(models_pred, positions):
    # Actual values
    fig.add_trace(
        go.Scatter(
            x=list(range(test_samples)),
            y=y_test[:test_samples, 0],  # First day of forecast
            name='Actual',
            mode='lines',
            line=dict(color='blue'),
            showlegend=(row == 1 and col == 1)
        ),
        row=row, col=col
    )

    # Predicted values
    fig.add_trace(
        go.Scatter(
            x=list(range(test_samples)),
            y=pred[:test_samples, 0],  # First day of forecast
            name='Predicted',
            mode='lines',
            line=dict(color='red', dash='dash'),
            showlegend=(row == 1 and col == 1)
        ),
        row=row, col=col
    )

fig.update_layout(
    title='Model Predictions vs Actual (1-Day Ahead Forecast)',
    height=800,
    template='plotly_white'
)

fig.show()

print("\n✅ Visualization complete!")

FORECAST VISUALIZATION



✅ Visualization complete!


## 10. Save Models and Predictions

In [17]:
# Save all models
print("=" * 60)
print("SAVING MODELS")
print("=" * 60)

lstm_model.save('/content/lstm_model.h5')
gru_model.save('/content/gru_model.h5')
tcn_model.save('/content/tcn_model.h5')
transformer_model.save('/content/transformer_model.h5')

print("\n✅ All models saved successfully!")

# Save predictions
predictions_df = pd.DataFrame({
    'LSTM_Pred': lstm_pred[:, 0],
    'GRU_Pred': gru_pred[:, 0],
    'TCN_Pred': tcn_pred[:, 0],
    'Transformer_Pred': transformer_pred[:, 0],
    'Actual': y_test[:, 0]
})

predictions_df.to_csv('/content/model_predictions.csv', index=False)
print("✅ Predictions saved to 'model_predictions.csv'")

SAVING MODELS

✅ All models saved successfully!
✅ Predictions saved to 'model_predictions.csv'
